In [ ]:
def getListedCode(typ="twse",codeLength=4):
    import sqlite3
    sqlite_file = "list.db" #上市上櫃下市 股票代號列表 資料庫檔案
    conn = sqlite3.connect(sqlite_file)
    c = conn.cursor()
    out = conn.execute("SELECT code, type FROM StockList WHERE type LIKE ? AND length(code) == ? ",("twse",codeLength)) 
    # 取 代號4碼 類別上市 股票代號
    codeList = []
    for i, row in enumerate(out):
        codeList.append(row[0])
    conn.close()
    print("取得股票代號總數:", len(codeList)) #codeList 為取出股票代號列表
    return codeList
print("getListedCode loaded")

codeList = getListedCode()

def getStrategyData(code,startdate,enddate,dbname="buysell.db",table="twse"):
    import sqlite3
    conn = sqlite3.connect(dbname)
    c = conn.cursor()
    out = conn.execute("SELECT * FROM twse WHERE code LIKE ? AND date between ? and ?",(code,startdate,enddate))
    dataList = []
    for i in out:
        dataList.append(i)
    conn.close()
    return dataList

dataList = getStrategyData("1101","2016-05-25","2017-05-25")
lastData = getStrategyData("1101","2017-05-24","2017-05-25","PriceVolData.db")
print("lastData:",lastData[0][6])
sellcount = 0
buycount = 0
selldata = []
buydata = []
for i in dataList:
    #print("Code:",i[1],"Open:",i[6],"Action:",i[8])
    if "sell" == i[8]:
        sellcount = sellcount + 1
        selldata.append([i[1],i[6],i[8]])
    if "buy" == i[8]:
        buycount = buycount + 1
        buydata.append([i[1],i[6],i[8]])

print("Buy:",buydata,"Sell:",selldata)
print("Buy:",buycount,"張 Sell:",sellcount,"張")

profit = 0
for buy,sell in zip(buydata,selldata):
    profit = profit + (sell[1] - buy[1])
    buydata.pop()
    selldata.pop()
    print("Profit:",profit,"Buy:",buy[1],"Sell:",sell[1])
if buycount > sellcount:
    for i in buydata:
        profit = profit + (lastData[0][6]-i[1])
        print("Profit:",profit,"Clear:",lastData[0][6],"Buy:",i[1])
print(profit)

In [ ]:
def getListedCode(typ="twse",codeLength=4):
    import sqlite3
    sqlite_file = "list.db" #上市上櫃下市 股票代號列表 資料庫檔案
    conn = sqlite3.connect(sqlite_file)
    c = conn.cursor()
    out = conn.execute("SELECT code, type FROM StockList WHERE type LIKE ? AND length(code) == ? ",("twse",codeLength)) 
    # 取 代號4碼 類別上市 股票代號
    codeList = []
    for i, row in enumerate(out):
        codeList.append(row[0])
    conn.close()
    print("取得股票代號總數:", len(codeList)) #codeList 為取出股票代號列表
    return codeList

def getStrategyData(code,startdate,enddate,dbname="buysell.db",table="twse"):
    import sqlite3
    conn = sqlite3.connect(dbname)
    c = conn.cursor()
    out = conn.execute("SELECT * FROM twse WHERE code LIKE ? AND date between ? and ?",(code,startdate,enddate))
    dataList = []
    for i in out:
        dataList.append(i)
    conn.close()
    return dataList

def getStats(dataList,lastData):
    sellcount = 0
    buycount = 0
    selldata = []
    buydata = []
    for i in dataList:
        #print("Code:",i[1],"Open:",i[6],"Action:",i[8])
        if "sell" == i[8]:
            sellcount = sellcount + 1
            selldata.append([i[1],i[6],i[8]])
        if "buy" == i[8]:
            buycount = buycount + 1
            buydata.append([i[1],i[6],i[8]])

    #print("Buy:",buydata,"Sell:",selldata)
    #print("Buy:",buycount,"張 Sell:",sellcount,"張")

    profit = 0
    win = 0
    lose = 0
    for buy,sell in zip(buydata,selldata):
        profit = profit + (sell[1] - buy[1])
        if 0 < (sell[1] - buy[1]):
            win = win + 1
        else:
            lose = lose + 1
        buydata.pop()
        selldata.pop()
        #print("Profit:",profit,"Buy:",buy[1],"Sell:",sell[1])
    if buycount > sellcount:
        for i in buydata:
            profit = profit + (lastData[0][6]-i[1])
            if 0 < (lastData[0][6]-i[1]):
                win = win + 1
            else:
                lose = lose + 1
            #print("Profit:",profit,"Clear:",lastData[0][6],"Buy:",i[1])
    return profit,win,lose

profitSum = 0
winSum = 0
loseSum = 0
codeList = getListedCode()
for code in codeList:
    dataList = getStrategyData(code,"2016-05-25","2017-05-25")
    lastData = getStrategyData(code,"2017-05-24","2017-05-25","PriceVolData.db")
    profit,win,lose = getStats(dataList,lastData)
    print("Code:",code,"Profit:",profit)
    profitSum = profitSum + profit
    winSum = winSum + win
    loseSum = loseSum + lose
print("Profit Sum:",profitSum,"Win Sum:",winSum,"Lose Sum:",loseSum)